In [1]:
import requests
import pandas as pd
import plotly.express as px

BASE_URL = "https://api.reccobeats.com/v1"







In [2]:
def get_tracks_metadata(ids: list):
    """Fetch track metadata. IDs can be Spotify or ReccoBeats IDs."""
    params = {"ids": ",".join(ids)}
    resp = requests.get(f"{BASE_URL}/track", params=params)
    resp.raise_for_status()
    return resp.json()

In [3]:
def get_audio_features(track_id: str):
    """Fetch audio features using ReccoBeats UUID (not Spotify ID)."""
    resp = requests.get(f"{BASE_URL}/track/{track_id}/audio-features")
    resp.raise_for_status()
    return resp.json()



In [4]:
def get_recommendations(seeds: list, size: int = 10):
    """Fetch recommendations based on seeds (can be Spotify IDs)."""
    params = {"seeds": ",".join(seeds), "size": size}
    resp = requests.get(f"{BASE_URL}/track/recommendation", params=params)
    resp.raise_for_status()
    return resp.json()


In [5]:
if __name__ == "__main__":
    # Example ReccoBeats ID (not Spotify)
    recco_track_id = "878dadea-33c5-4c08-bdb9-e2b117475a99"

    # Step 1: Get metadata -> includes ReccoBeats ID
    metadata = get_tracks_metadata([recco_track_id])
    print("Metadata response:", metadata)

    if metadata.get("content"):
        track_info = metadata["content"][0]   # ✅ correct key
        recco_id = track_info["id"]
        print("Track:", track_info.get("name", "Unknown"), "| ReccoBeats ID:", recco_id)

        # Step 2: Get audio features using ReccoBeats ID
        features = get_audio_features(recco_id)
        print("Audio Features:", features, "\n")

        # Step 3: Get recommendations
        recs = get_recommendations([recco_track_id], size=5)
        print("Recommendations response:", recs)  # 👀 debug

        # Build DataFrame
        data = []
        # Try both "tracks" and "content"
        for t in recs.get("tracks", recs.get("content", [])):
            f = get_audio_features(t["id"])
            data.append({
                "Track": t.get("name", "Unknown"),
                "Artist": ", ".join([a["name"] for a in t.get("artists", [])]),
                "Danceability": f.get("danceability"),
                "Tempo": f.get("tempo"),
                "Energy": f.get("energy"),
                "Popularity": t.get("popularity", None)
            })
            df = pd.DataFrame(data)
        print("✅ DataFrame created:")
        print(df.head())

        if df.empty:
            print("⚠️ No recommendations or features available → nothing to plot")
        else:
            fig = px.scatter(
                df,
                x="Tempo",
                y="Danceability",
                size="Energy",
                color="Artist",
                hover_name="Track",
                title="Danceability vs Tempo of Recommended Songs"
            )
            import plotly.io as pio
            pio.renderers.default = "browser"
            fig.show()  # ✅ properly indented

    else:
        print("❌ No metadata found for this track ID")



Metadata response: {'content': [{'id': '878dadea-33c5-4c08-bdb9-e2b117475a99', 'trackTitle': 'All Too Well', 'artists': [{'id': 'c7b330b5-a62e-420c-bf02-943ca6bb8746', 'name': 'Taylor Swift', 'href': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'}], 'durationMs': 329160, 'isrc': 'USCJY1231021', 'ean': None, 'upc': None, 'href': 'https://open.spotify.com/track/00vJzaoxM3Eja1doBUhX0P', 'availableCountries': 'AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,DE,EC,EE,SV,FI,FR,GR,GT,HN,HK,HU,IS,IE,IT,LV,LT,LU,MY,MT,MX,NL,NZ,NI,NO,PA,PY,PE,PH,PL,PT,SG,SK,ES,SE,CH,TW,TR,UY,US,GB,AD,LI,MC,ID,JP,TH,VN,RO,IL,ZA,SA,AE,BH,QA,OM,KW,EG,MA,DZ,TN,LB,JO,PS,IN,BY,KZ,MD,UA,AL,BA,HR,ME,MK,RS,SI,KR,BD,PK,LK,GH,KE,NG,TZ,UG,AG,AM,BS,BB,BZ,BT,BW,BF,CV,CW,DM,FJ,GM,GE,GD,GW,GY,HT,JM,KI,LS,LR,MW,MV,ML,MH,FM,NA,NR,NE,PW,PG,WS,SM,ST,SN,SC,SL,SB,KN,LC,VC,SR,TL,TO,TT,TV,VU,AZ,BN,BI,KH,CM,TD,KM,GQ,SZ,GA,GN,KG,LA,MO,MR,MN,NP,RW,TG,UZ,ZW,BJ,MG,MU,MZ,AO,CI,DJ,ZM,CD,CG,IQ,LY,TJ,VE,ET,XK', 'popularity': 34}]}
Track:

In [7]:
df.to_csv("spotify_playlist_data.csv", index=False)